##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introducción al seccionamiento de tensores

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/tensor_slicing"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/tensor_slicing.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/tensor_slicing.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/tensor_slicing.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

Cuando trabajamos en aplicaciones de aprendizaje automático como la detección de objetos y el NPL (procesamiento de lenguaje natural), a veces, es necesario trabajar con subsecciones (<em>slices</em>) de tensores. Por ejemplo, si la arquitectura de su modelo incluye el enrutamiento, en que una capa podría controlar qué ejemplo de entrenamiento se enruta a la siguiente capa. En tal caso, podría usar las <em>ops</em> (operaciones) de seccionamiento de tensores para dividir los tensores y volver a reunirlos en el orden correcto.

En aplicaciones de procesamiento de lenguaje natural, se puede usar el seccionamiento de tensores para el enmascaramiento de palabras durante el entrenamiento. Por ejemplo, es posible generar datos de entrenamiento a partir de una lista de sentencias mediante la selección de un índice de palabras para enmascarar en cada sentencia, tomando la palabra como etiqueta y, después, reemplazando la palabra elegida por un token de máscara.

En esta guía, aprenderá a usar las API de TensorFlow para lo siguiente:

- Extraer secciones (<em>slices</em>) de un tensor
- Insertar datos a índices específicos en un tensor

En esta guía se sobreentiende que el lector está familiarizado con la indexación de tensores. Antes de empezar a leer esta guía, consulte las secciones sobre indexación de las guías sobre [Tensores](https://www.tensorflow.org/guide/tensor#indexing) y [TensorFlow NumPy](https://www.tensorflow.org/guide/tf_numpy#indexing).

## Preparación


In [ ]:
import tensorflow as tf
import numpy as np

## Extracción de secciones de tensores

Realice un seccionamiento de los tensores tipo NumPy con `tf.slice`.


In [ ]:
t1 = tf.constant([0, 1, 2, 3, 4, 5, 6, 7])

print(tf.slice(t1,
               begin=[1],
               size=[3]))

Como alternativa, se puede usar más sintaxis Pythonic. Tenga en cuenta que los espacios de separación de las secciones del tensor son los mismos a lo largo de un rango desde el inicio hasta que se detiene.

In [ ]:
print(t1[1:4])

<img src="images/tf_slicing/slice_1d_1.png">

In [ ]:
print(t1[-3:])

<img src="images/tf_slicing/slice_1d_2.png">

Con los tensores bidimensionales, se puede usar algo como lo siguiente:

In [ ]:
t2 = tf.constant([[0, 1, 2, 3, 4],
                  [5, 6, 7, 8, 9],
                  [10, 11, 12, 13, 14],
                  [15, 16, 17, 18, 19]])

print(t2[:-1, 1:3])

<img src="images/tf_slicing/slice_2d_1.png">

Con tensores dimensionales más altos se puede usar `tf.slice`.

In [ ]:
t3 = tf.constant([[[1, 3, 5, 7],
                   [9, 11, 13, 15]],
                  [[17, 19, 21, 23],
                   [25, 27, 29, 31]]
                  ])

print(tf.slice(t3,
               begin=[1, 1, 0],
               size=[1, 1, 2]))

También es posible usar `tf.strided_slice` para extraer secciones de tensores mediante el "<em>striding</em>" (desplazamiento) sobre las dimensiones del tensor.

Use `tf.gather` para extraer índices específicos a partir de un solo eje de un tensor.

In [ ]:
print(tf.gather(t1,
                indices=[0, 3, 6]))

# This is similar to doing

t1[::3]

<img src="images/tf_slicing/slice_1d_3.png">

No se requieren índices para lograr el espaciamiento parejo de `tf.gather`.

In [ ]:
alphabet = tf.constant(list('abcdefghijklmnopqrstuvwxyz'))

print(tf.gather(alphabet,
                indices=[2, 0, 19, 18]))

<img src="images/tf_slicing/gather_1.png">

Para extraer secciones de múltiples ejes de un tensor, use `tf.gather_nd`. Esto resulta útil cuando lo que desea es reunir los elementos de una matriz en oposición a sus filas o columnas.

In [ ]:
t4 = tf.constant([[0, 5],
                  [1, 6],
                  [2, 7],
                  [3, 8],
                  [4, 9]])

print(tf.gather_nd(t4,
                   indices=[[2], [3], [0]]))

<img src="images/tf_slicing/gather_2.png">

In [ ]:
t5 = np.reshape(np.arange(18), [2, 3, 3])

print(tf.gather_nd(t5,
                   indices=[[0, 0, 0], [1, 2, 1]]))

In [ ]:
# Return a list of two matrices

print(tf.gather_nd(t5,
                   indices=[[[0, 0], [0, 2]], [[1, 0], [1, 2]]]))

In [ ]:
# Return one matrix

print(tf.gather_nd(t5,
                   indices=[[0, 0], [0, 2], [1, 0], [1, 2]]))

## Inserción de datos en tensores

Use `tf.scatter_nd` para insertar datos en secciones o índices específicos de un tensor. Tenga en cuenta que el tensor en el que inserte valores se inicializará a cero.

In [ ]:
t6 = tf.constant([10])
indices = tf.constant([[1], [3], [5], [7], [9]])
data = tf.constant([2, 4, 6, 8, 10])

print(tf.scatter_nd(indices=indices,
                    updates=data,
                    shape=t6))

Los métodos como `tf.scatter_nd` para los que se requieren tensores inicializados en cero son similares a los de los inicializadores de tensores dispersos. Se pueden usar `tf.gather_nd` y `tf.scatter_nd` para imitar el comportamiento de las operaciones de tensores dispersos.

Considere un ejemplo de construcción de un tensor disperso con estos dos métodos juntos.

In [ ]:
# Gather values from one tensor by specifying indices

new_indices = tf.constant([[0, 2], [2, 1], [3, 3]])
t7 = tf.gather_nd(t2, indices=new_indices)

<img src="images/tf_slicing/gather_nd_sparse.png">

In [ ]:
# Add these values into a new tensor

t8 = tf.scatter_nd(indices=new_indices, updates=t7, shape=tf.constant([4, 5]))

print(t8)

Es similar a:

In [ ]:
t9 = tf.SparseTensor(indices=[[0, 2], [2, 1], [3, 3]],
                     values=[2, 11, 18],
                     dense_shape=[4, 5])

print(t9)

In [ ]:
# Convert the sparse tensor into a dense tensor

t10 = tf.sparse.to_dense(t9)

print(t10)

Para insertar datos en un tensor con valores preexistentes, use `tf.tensor_scatter_nd_add`.

In [ ]:
t11 = tf.constant([[2, 7, 0],
                   [9, 0, 1],
                   [0, 3, 8]])

# Convert the tensor into a magic square by inserting numbers at appropriate indices

t12 = tf.tensor_scatter_nd_add(t11,
                               indices=[[0, 2], [1, 1], [2, 0]],
                               updates=[6, 5, 4])

print(t12)

De un modo similar, use `tf.tensor_scatter_nd_sub` para restar valores de un tensor con valores preexistentes.

In [ ]:
# Convert the tensor into an identity matrix

t13 = tf.tensor_scatter_nd_sub(t11,
                               indices=[[0, 0], [0, 1], [1, 0], [1, 1], [1, 2], [2, 1], [2, 2]],
                               updates=[1, 7, 9, -1, 1, 3, 7])

print(t13)

Use `tf.tensor_scatter_nd_min` para copiar de un tensor a otro valores mínimos entre elementos correspondientes (<em>element-wise</em>).

In [ ]:
t14 = tf.constant([[-2, -7, 0],
                   [-9, 0, 1],
                   [0, -3, -8]])

t15 = tf.tensor_scatter_nd_min(t14,
                               indices=[[0, 2], [1, 1], [2, 0]],
                               updates=[-6, -5, -4])

print(t15)

Asimismo, use `tf.tensor_scatter_nd_max` para copiar de un tensor a otro valores máximos entre elementos correspondientes (<em>element-wise</em>).

In [ ]:
t16 = tf.tensor_scatter_nd_max(t14,
                               indices=[[0, 2], [1, 1], [2, 0]],
                               updates=[6, 5, 4])

print(t16)

## Lecturas y recursos complementarios

Con esta guía, ha aprendido a usar las operaciones de seccionamiento de tensores disponibles en TensorFlow para ejercer un control más detallado de los elementos de los tensores.

- Revise las operaciones de seccionamiento disponibles con TensorFlow NumPy tales como `tf.experimental.numpy.take_along_axis` y `tf.experimental.numpy.take`.

- También consulte la [guía sobre tensores](https://www.tensorflow.org/guide/tensor) y la [guía sobre variables](https://www.tensorflow.org/guide/variable).